In [ ]:
# Install dependencies
!pip install -q -U torch torchvision transformers peft bitsandbytes accelerate datasets trl qwen-vl-utils

import os
import torch
import warnings
import pandas as pd
from PIL import Image
from google.colab import drive
from sklearn.model_selection import train_test_split
from transformers import Qwen2VLForConditionalGeneration, Qwen2VLProcessor, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
from trl import SFTConfig, SFTTrainer

warnings.filterwarnings("ignore")

# Mount Drive
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

print(f"✅ Setup complete. Device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

In [ ]:
# --- CONFIGURATION ---
MODEL_ID = "Qwen/Qwen2-VL-2B-Instruct"

# Paths (Update these paths for your environment)
DATASET_PATH = "/content/drive/MyDrive/yemek/yemek_dataset_v3_corrected.csv"
IMAGE_DIR = "/content/drive/MyDrive/yemek/"
OUTPUT_DIR = "/content/drive/MyDrive/outputs_v3_expert"

# Training Hyperparameters
EPOCHS = 10
LEARNING_RATE = 2e-4
BATCH_SIZE = 8
GRAD_ACCUMULATION = 2
LORA_RANK = 64
LORA_ALPHA = 128

In [ ]:
# Quantization Config (4-bit)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load Base Model
model = Qwen2VLForConditionalGeneration.from_pretrained(
    MODEL_ID,
    device_map="auto",
    quantization_config=bnb_config,
    trust_remote_code=True
)

# Load Processor
processor = Qwen2VLProcessor.from_pretrained(MODEL_ID, trust_remote_code=True)

In [ ]:
# System Prompt for the AI Dietitian role
SYSTEM_PROMPT = "Sen Türk mutfağı konusunda uzman bir diyetisyen ve VLM asistanısın. Yemekleri tanı ve besin değerlerini söyle."

def format_data(sample):
    """Formats the dataset into Qwen2-VL conversation structure."""
    try:
        image_path = os.path.join(IMAGE_DIR, sample["image_path"])
        image = Image.open(image_path)

        return [
            {"role": "system", "content": [{"type": "text", "text": SYSTEM_PROMPT}]},
            {"role": "user", "content": [{"type": "image", "image": image}, {"type": "text", "text": sample["question"]}]},
            {"role": "assistant", "content": [{"type": "text", "text": sample["answer"]}]}
        ]
    except Exception as e:
        return None

# Load and Split Data
df = pd.read_csv(DATASET_PATH)
train_df, eval_df = train_test_split(df, test_size=0.05, random_state=42)

# Create Datasets
train_dataset = [res for x in train_df.to_dict('records') if (res := format_data(x))]
eval_dataset = [res for x in eval_df.to_dict('records') if (res := format_data(x))]

print(f"Train Size: {len(train_dataset)} | Eval Size: {len(eval_dataset)}")

In [ ]:
# LoRA Configuration
peft_config = LoraConfig(
    lora_alpha=LORA_ALPHA,
    lora_dropout=0.05,
    r=LORA_RANK,
    bias="none",
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

# Apply LoRA
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# Data Collator
def collate_fn(examples):
    texts = [processor.apply_chat_template(ex, tokenize=False, add_generation_prompt=False) for ex in examples]
    images = [ex[1]["content"][0]["image"] for ex in examples]

    batch = processor(text=texts, images=images, return_tensors="pt", padding=True)
    batch["labels"] = batch["input_ids"].clone()
    return batch

# Training Arguments
training_args = SFTConfig(
    output_dir=OUTPUT_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACCUMULATION,
    learning_rate=LEARNING_RATE,
    save_steps=500,
    logging_steps=10,
    bf16=True,
    dataset_kwargs={"skip_prepare_dataset": True},
    report_to="none"
)

In [ ]:
# Initialize Trainer
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=collate_fn,
    processing_class=processor.tokenizer,
)

# Start Training
print("🚀 Starting training...")
trainer.train()

# Save Model
trainer.save_model(OUTPUT_DIR)
print(f"✅ Model saved to {OUTPUT_DIR}")

In [ ]:
import matplotlib.pyplot as plt

def run_inference(image_path, question="Bu yemek nedir?"):
    """Runs inference on a single image."""
    image = Image.open(image_path)

    conversation = [
        {"role": "system", "content": [{"type": "text", "text": SYSTEM_PROMPT}]},
        {"role": "user", "content": [{"type": "image", "image": image}, {"type": "text", "text": question}]}
    ]

    text_prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)
    inputs = processor(text=[text_prompt], images=[image], return_tensors="pt").to("cuda")

    output_ids = model.generate(**inputs, max_new_tokens=128)
    response = processor.batch_decode(output_ids, skip_special_tokens=True)[0]

    # Display
    plt.imshow(image)
    plt.axis('off')
    plt.show()
    print(f"Answer: {response.split('assistant')[-1].strip()}")

# Example Usage (Update path)
# run_inference("/content/drive/MyDrive/test_images/kebab.jpg")